## 1. 데이터 전처리

In [10]:
%pip install "torch==2.8.0"
%pip install "torchvision==0.23.0" 
%pip install "torchaudio==2.8.0"
%pip install "transformers==4.55.2"
%pip install "tokenizers==0.21.4"
%pip install "safetensors==0.6.2"
%pip install "huggingface-hub==0.34.4"
%pip install "datasets==4.0.0"
%pip install "accelerate==1.10.0"
%pip install "trl==0.21.0"
%pip install "peft==0.17.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 MB 35.6 MB/s  0:00:09m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 120.7 MB/s  0:00:030:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 119.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 177.4 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 29.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 122.6 MB/s  0:00:030:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 173.9 MB/s  0:00:010:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 138.4 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 172.4 MB/s  0:00:010:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 173.2 MB/s  0:00:010:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.4/322.4 MB 119.1 MB/s  0:00:020:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [11]:
%pip install hf_transfer

Note: you may need to restart the kernel to use updated packages.


In [1]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

빠른 학습을 위해 학습 데이터와 테스트 데이터를 2:8 비율로 분할합니다. 이 값을 변경하고자 하는 분은 test_ratio의 값을 변경하세요.

In [2]:
from datasets import load_dataset
import random

# 1. 허깅페이스 허브에서 데이터셋 로드
dataset = load_dataset("iamjoon/manufacturing-text-to-sql", split="train")

print(f"원본 데이터 개수: {len(dataset)}")

# 2. train/test split
test_ratio = 0.2

indices = list(range(len(dataset)))
random.shuffle(indices)

test_size = int(len(indices) * test_ratio)

test_indices = indices[:test_size]
train_indices = indices[test_size:]

# 3. OpenAI format 변환 함수
def format_data(sample):
    return {
        "messages": [
            {"role": "system", "content": sample["system_prompt"]},
            {"role": "user", "content": sample["user_prompt"]},
            {"role": "assistant", "content": sample["assistant"]},
        ]
    }

# 4. 변환
train_dataset = [format_data(dataset[i]) for i in train_indices]
test_dataset = [format_data(dataset[i]) for i in test_indices]

# 5. 결과 출력
print(f"Train: {len(train_dataset)}개")
print(f"Test:  {len(test_dataset)}개")

원본 데이터 개수: 776
Train: 621개
Test:  155개


In [3]:
train_dataset[345]["messages"]

[{'role': 'system',
  'content': "\n당신은 SQL을 생성하는 AI 모델입니다.\n아래는 데이터베이스 스키마(DDL)입니다.\n\n<SCHEMA>\n-- 작업완료로그 테이블\nCREATE TABLE IF NOT EXISTS `log_lottranslog` (\n  `TRANSLOGID` bigint(20) NOT NULL DEFAULT 0 COMMENT '로그ID',\n  `LOTNO` char(20) NOT NULL DEFAULT '' COMMENT 'LOT번호',\n  `LINENO` char(20) DEFAULT NULL COMMENT '생산라인번호',\n  `TRANSACTIONNAME` char(50) NOT NULL DEFAULT '' COMMENT '처리명',\n  `TIMELOGGED` datetime(3) NOT NULL DEFAULT '0000-00-00 00:00:00.000' COMMENT '로그입력일시',\n  `ACTUALTIME` datetime(3) NOT NULL DEFAULT '0000-00-00 00:00:00.000' COMMENT '실제실행일시',\n  `MATERIALCODE` char(30) NOT NULL DEFAULT '' COMMENT '자재코드',\n  `MATERIALNAME` char(50) NOT NULL DEFAULT '' COMMENT '자재명',\n  `TRANSQTY` double NOT NULL DEFAULT 0 COMMENT '변경수량',\n  `CURRENTQTY` double NOT NULL DEFAULT 0 COMMENT '현재수량',\n  `NEXTQTY` double DEFAULT NULL COMMENT '변경반영된수량',\n  `TRANSUOM` char(5) DEFAULT NULL COMMENT '측정단위',\n  `WAREHOUSECODE` char(20) DEFAULT NULL COMMENT '창고코드',\n  `BOPMATERIALCODE` char(3

In [4]:
# 리스트 형태에서 다시 Dataset 객체로 변경
print(type(train_dataset))
print(type(test_dataset))
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)
print(type(train_dataset))
print(type(test_dataset))

<class 'list'>
<class 'list'>
<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [5]:
train_dataset[0]

{'messages': [{'content': "\n당신은 SQL을 생성하는 AI 모델입니다.\n아래는 데이터베이스 스키마(DDL)입니다.\n\n<SCHEMA>\n-- 작업완료로그 테이블\nCREATE TABLE IF NOT EXISTS `log_lottranslog` (\n  `TRANSLOGID` bigint(20) NOT NULL DEFAULT 0 COMMENT '로그ID',\n  `LOTNO` char(20) NOT NULL DEFAULT '' COMMENT 'LOT번호',\n  `LINENO` char(20) DEFAULT NULL COMMENT '생산라인번호',\n  `TRANSACTIONNAME` char(50) NOT NULL DEFAULT '' COMMENT '처리명',\n  `TIMELOGGED` datetime(3) NOT NULL DEFAULT '0000-00-00 00:00:00.000' COMMENT '로그입력일시',\n  `ACTUALTIME` datetime(3) NOT NULL DEFAULT '0000-00-00 00:00:00.000' COMMENT '실제실행일시',\n  `MATERIALCODE` char(30) NOT NULL DEFAULT '' COMMENT '자재코드',\n  `MATERIALNAME` char(50) NOT NULL DEFAULT '' COMMENT '자재명',\n  `TRANSQTY` double NOT NULL DEFAULT 0 COMMENT '변경수량',\n  `CURRENTQTY` double NOT NULL DEFAULT 0 COMMENT '현재수량',\n  `NEXTQTY` double DEFAULT NULL COMMENT '변경반영된수량',\n  `TRANSUOM` char(5) DEFAULT NULL COMMENT '측정단위',\n  `WAREHOUSECODE` char(20) DEFAULT NULL COMMENT '창고코드',\n  `BOPMATERIALCODE` char(30) DEFA

## 2. 모델 로드 및 템플릿 적용

In [6]:
# 허깅페이스 모델 ID
model_id = "Qwen/Qwen3-4B-Instruct-2507" 

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [7]:
# 템플릿 적용
text = tokenizer.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|im_start|>system

당신은 SQL을 생성하는 AI 모델입니다.
아래는 데이터베이스 스키마(DDL)입니다.

<SCHEMA>
-- 작업완료로그 테이블
CREATE TABLE IF NOT EXISTS `log_lottranslog` (
  `TRANSLOGID` bigint(20) NOT NULL DEFAULT 0 COMMENT '로그ID',
  `LOTNO` char(20) NOT NULL DEFAULT '' COMMENT 'LOT번호',
  `LINENO` char(20) DEFAULT NULL COMMENT '생산라인번호',
  `TRANSACTIONNAME` char(50) NOT NULL DEFAULT '' COMMENT '처리명',
  `TIMELOGGED` datetime(3) NOT NULL DEFAULT '0000-00-00 00:00:00.000' COMMENT '로그입력일시',
  `ACTUALTIME` datetime(3) NOT NULL DEFAULT '0000-00-00 00:00:00.000' COMMENT '실제실행일시',
  `MATERIALCODE` char(30) NOT NULL DEFAULT '' COMMENT '자재코드',
  `MATERIALNAME` char(50) NOT NULL DEFAULT '' COMMENT '자재명',
  `TRANSQTY` double NOT NULL DEFAULT 0 COMMENT '변경수량',
  `CURRENTQTY` double NOT NULL DEFAULT 0 COMMENT '현재수량',
  `NEXTQTY` double DEFAULT NULL COMMENT '변경반영된수량',
  `TRANSUOM` char(5) DEFAULT NULL COMMENT '측정단위',
  `WAREHOUSECODE` char(20) DEFAULT NULL COMMENT '창고코드',
  `BOPMATERIALCODE` char(30) DEFAULT NULL COMMENT '자재코드',
  `

## 3. LoRA와 SFTConfig 설정

In [10]:
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

`lora_alpha`: LoRA(Low-Rank Adaptation)에서 사용하는 스케일링 계수를 설정합니다. LoRA의 가중치 업데이트가 모델에 미치는 영향을 조정하는 역할을 하며, 일반적으로 학습 안정성과 관련이 있습니다.

`lora_dropout`: LoRA 적용 시 드롭아웃 확률을 설정합니다. 드롭아웃은 과적합(overfitting)을 방지하기 위해 일부 뉴런을 랜덤하게 비활성화하는 정규화 기법입니다. `0.1`로 설정하면 학습 중 10%의 뉴런이 비활성화됩니다.

`r`: LoRA의 랭크(rank)를 설정합니다. 이는 LoRA가 학습할 저차원 공간의 크기를 결정합니다. 작은 값일수록 계산 및 메모리 효율이 높아지지만 모델의 학습 능력이 제한될 수 있습니다.

`bias`: LoRA 적용 시 편향(bias) 처리 방식을 지정합니다. `"none"`으로 설정하면 편향이 LoRA에 의해 조정되지 않습니다. `"all"` 또는 `"lora_only"`와 같은 값으로 변경하여 편향을 조정할 수도 있습니다.

`target_modules`: LoRA를 적용할 특정 모듈(레이어)의 이름을 리스트로 지정합니다. 예제에서는 `"q_proj"`와 `"v_proj"`를 지정하여, 주로 Self-Attention 메커니즘의 쿼리와 값 프로젝션 부분에 LoRA를 적용합니다.

`task_type`: LoRA가 적용되는 작업 유형을 지정합니다. `"CAUSAL_LM"`은 Causal Language Modeling, 즉 시퀀스 생성 작업에 해당합니다. 다른 예로는 `"SEQ2SEQ_LM"`(시퀀스-투-시퀀스 언어 모델링) 등이 있습니다.

In [19]:
args = SFTConfig(
    output_dir="qwen3-4b-text-to-sql-ko",           # 저장될 디렉토리와 저장소 ID
    num_train_epochs=5,                      # 학습할 총 에포크 수 
    per_device_train_batch_size=2,           # GPU당 배치 크기
    gradient_accumulation_steps=2,           # 그래디언트 누적 스텝 수
    gradient_checkpointing=True,             # 메모리 절약을 위한 체크포인팅
    optim="adamw_torch_fused",               # 최적화기
    logging_steps=10,                        # 로그 기록 주기
    save_strategy="steps",                   # 저장 전략
    save_steps=50,                           # 저장 주기
    bf16=True,                              # bfloat16 사용
    learning_rate=1e-4,                     # 학습률
    max_grad_norm=0.3,                      # 그래디언트 클리핑
    warmup_ratio=0.03,                      # 워밍업 비율
    lr_scheduler_type="constant",           # 고정 학습률
    push_to_hub=False,                      # 허브 업로드 안 함
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    report_to=None,
    max_length=max_seq_length,              # 최대 시퀀스 길이 추가
)

`output_dir`: 학습 결과가 저장될 디렉토리 또는 모델 저장소의 이름을 지정합니다. 이 디렉토리에 학습된 모델 가중치, 설정 파일, 로그 파일 등이 저장됩니다.

`num_train_epochs`: 모델을 학습시키는 총 에포크(epoch) 수를 지정합니다. 에포크는 학습 데이터 전체를 한 번 순회한 주기를 의미합니다. 예를 들어, `3`으로 설정하면 데이터셋을 3번 학습합니다.

`per_device_train_batch_size`: GPU 한 대당 사용되는 배치(batch)의 크기를 설정합니다. 배치 크기는 모델이 한 번에 처리하는 데이터 샘플의 수를 의미합니다. 작은 크기는 메모리 사용량이 적지만 학습 시간이 증가할 수 있습니다.

`gradient_accumulation_steps`: 그래디언트를 누적할 스텝(step) 수를 지정합니다. 이 값이 2로 설정된 경우, 두 스텝마다 그래디언트를 업데이트합니다. 배치 크기를 가상으로 늘리는 효과가 있으며, GPU 메모리 부족 문제를 해결할 때 유용합니다.

`gradient_checkpointing`: 그래디언트 체크포인팅을 활성화하여 메모리를 절약합니다. 이 옵션은 계산 그래프를 일부 저장하지 않고 다시 계산하여 메모리를 절약하지만, 속도가 약간 느려질 수 있습니다.

`optim`: 학습 시 사용할 최적화 알고리즘을 설정합니다. `adamw_torch_fused`는 PyTorch의 효율적인 AdamW 최적화기를 사용합니다.

`logging_steps`: 로그를 기록하는 주기를 스텝 단위로 지정합니다. 예를 들어, `10`으로 설정하면 매 10 스텝마다 로그를 기록합니다.

`save_strategy`: 모델을 저장하는 전략을 설정합니다. `"steps"`로 설정된 경우, 지정된 스텝마다 모델이 저장됩니다.

`save_steps`: 모델을 저장하는 주기를 스텝 단위로 설정합니다. 예를 들어, `50`으로 설정하면 매 50 스텝마다 모델을 저장합니다.

`bf16`: bfloat16 정밀도를 사용하도록 설정합니다. bfloat16은 FP32와 유사한 범위를 제공하면서 메모리와 계산 효율성을 높입니다.

`learning_rate`: 학습률을 지정합니다. 학습률은 모델의 가중치가 한 번의 업데이트에서 얼마나 크게 변할지를 결정합니다. 일반적으로 작은 값을 사용하여 안정적인 학습을 유도합니다.

`max_grad_norm`: 그래디언트 클리핑의 임계값을 설정합니다. 이 값보다 큰 그래디언트가 발생하면, 임계값으로 조정하여 폭발적 그래디언트를 방지합니다.

`warmup_ratio`: 학습 초기 단계에서 학습률을 선형으로 증가시키는 워밍업 비율을 지정합니다. 학습의 안정성을 높이기 위해 사용됩니다.

`lr_scheduler_type`: 학습률 스케줄러의 유형을 설정합니다. `"constant"`는 학습률을 일정하게 유지합니다.

`push_to_hub`: 학습된 모델을 허브에 업로드할지 여부를 설정합니다. `False`로 설정하면 업로드하지 않습니다.

`remove_unused_columns`: 사용되지 않는 열을 제거할지 여부를 설정합니다. True로 설정하면 메모리를 절약할 수 있습니다.

`dataset_kwargs`: 데이터셋 로딩 시 추가적인 설정을 전달합니다. 예제에서는 `skip_prepare_dataset: True`로 설정하여 데이터셋 준비 단계를 건너뜹니다.

`report_to`: 학습 로그를 보고할 대상을 지정합니다. `None`으로 설정되면 로그가 기록되지 않습니다.

## 4. 학습 중 전처리 함수: collate_fn

In [20]:
def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }
    
    for example in batch:
        # messages의 각 내용에서 개행문자 제거
        clean_messages = []
        for message in example["messages"]:
            clean_message = {
                "role": message["role"],
                "content": message["content"]
            }
            clean_messages.append(clean_message)
        
        # 깨끗해진 메시지로 템플릿 적용
        text = tokenizer.apply_chat_template(
            clean_messages,
            tokenize=False,
            add_generation_prompt=False
        ).strip()
        
        # 텍스트를 토큰화
        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=max_seq_length,
            padding=False,
            return_tensors=None,
        )
        
        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]
        
        # 레이블 초기화
        labels = [-100] * len(input_ids)
        
        # assistant 응답 부분 찾기
        im_start = "<|im_start|>"
        im_end = "<|im_end|>"
        assistant = "assistant"
        
        # 토큰 ID 가져오기
        im_start_tokens = tokenizer.encode(im_start, add_special_tokens=False)
        im_end_tokens = tokenizer.encode(im_end, add_special_tokens=False)
        assistant_tokens = tokenizer.encode(assistant, add_special_tokens=False)
        
        i = 0
        while i < len(input_ids):
            # <|im_start|>assistant 찾기
            if (i + len(im_start_tokens) <= len(input_ids) and 
                input_ids[i:i+len(im_start_tokens)] == im_start_tokens):
                
                # assistant 토큰 찾기
                assistant_pos = i + len(im_start_tokens)
                if (assistant_pos + len(assistant_tokens) <= len(input_ids) and 
                    input_ids[assistant_pos:assistant_pos+len(assistant_tokens)] == assistant_tokens):
                    
                    # assistant 응답의 시작 위치로 이동
                    current_pos = assistant_pos + len(assistant_tokens)
                    
                    # <|im_end|>를 찾을 때까지 레이블 설정
                    while current_pos < len(input_ids):
                        if (current_pos + len(im_end_tokens) <= len(input_ids) and 
                            input_ids[current_pos:current_pos+len(im_end_tokens)] == im_end_tokens):
                            # <|im_end|> 토큰도 레이블에 포함
                            for j in range(len(im_end_tokens)):
                                labels[current_pos + j] = input_ids[current_pos + j]
                            break
                        labels[current_pos] = input_ids[current_pos]
                        current_pos += 1
                    
                    i = current_pos
                
            i += 1
        
        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)
    
    # 패딩 적용
    max_length = max(len(ids) for ids in new_batch["input_ids"])
    
    for i in range(len(new_batch["input_ids"])):
        padding_length = max_length - len(new_batch["input_ids"][i])
        
        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * padding_length)
        new_batch["attention_mask"][i].extend([0] * padding_length)
        new_batch["labels"][i].extend([-100] * padding_length)
    
    # 텐서로 변환
    for k, v in new_batch.items():
        new_batch[k] = torch.tensor(v)
    
    return new_batch

`collate_fn(batch)` 함수는 자연어 처리 모델 학습을 위해 데이터를 전처리하는 역할을 수행합니다. 이 함수는 배치 내의 데이터를 처리하여 모델이 사용할 수 있는 입력 형식으로 변환합니다.

먼저, 각 샘플의 메시지에서 개행 문자를 제거하고 필요한 정보만 남깁니다. 정리된 메시지로 텍스트를 구성하고 이를 토큰화하여 input_ids와 attention_mask를 생성합니다. 이후 레이블 데이터를 초기화한 다음, 특정 토큰 패턴(<|im_start|>assistant 이후부터 <|im_end|>까지)을 찾아 해당 범위에 레이블을 설정합니다. 이 범위를 제외한 나머지 위치는 -100으로 설정하여 손실 계산에서 제외되도록 합니다.

최종적으로, 배치 내 모든 샘플의 길이를 동일하게 맞추기 위해 패딩 작업을 수행합니다. 이 과정에서 입력 데이터에는 패딩 토큰 ID를 추가하고, 어텐션 마스크에는 0을 추가하며, 레이블에는 -100을 추가합니다. 모든 데이터는 PyTorch 텐서로 변환되어 반환됩니다.

In [21]:
# 최대 길이
max_seq_length=8192

# collate_fn 테스트 (배치 크기 1로)
example = train_dataset[0]
batch = collate_fn([example])

print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 2877])
어텐션 마스크 형태: torch.Size([1, 2877])
레이블 형태: torch.Size([1, 2877])


In [22]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0].tolist())

입력에 대한 정수 인코딩 결과:
[151644, 8948, 271, 64795, 82528, 33704, 7870, 17877, 51917, 42905, 15235, 54070, 142713, 78952, 624, 52959, 53442, 16560, 54248, 132947, 12802, 24897, 79207, 44680, 92120, 125544, 7, 58781, 8, 78952, 382, 27, 3540, 35839, 397, 313, 139495, 130973, 63256, 81650, 10764, 72509, 12802, 135032, 198, 22599, 14363, 11551, 4183, 34839, 1565, 839, 91849, 1458, 839, 63, 2399, 220, 1565, 43103, 7243, 915, 63, 79663, 7, 17, 15, 8, 4183, 1770, 11955, 220, 15, 50505, 364, 81650, 915, 751, 220, 1565, 15745, 8996, 63, 1161, 7, 17, 15, 8, 4183, 1770, 11955, 3355, 50505, 364, 15745, 72048, 751, 220, 1565, 63127, 63362, 63, 1161, 7, 17, 15, 8, 11955, 1770, 50505, 364, 76435, 85057, 50340, 31328, 72048, 751, 220, 1565, 48773, 46159, 7535, 63, 1161, 7, 20, 15, 8, 4183, 1770, 11955, 3355, 50505, 364, 131342, 79632, 751, 220, 1565, 18129, 7243, 45522, 63, 8874, 7, 18, 8, 4183, 1770, 11955, 364, 15, 15, 15, 15, 12, 15, 15, 12, 15, 15, 220, 15, 15, 25, 15, 15, 25, 15, 15, 13, 15, 15, 15, 6, 

In [23]:
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0].tolist())

레이블에 대한 정수 인코딩 결과:
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -1

LLM 학습에서 `input_ids`와 `labels`는 모델의 학습 목표에 따라 생성됩니다. 이를 예시 문장과 정수 인코딩을 통해 상세히 설명하겠습니다.


예를 들어, 다음과 같은 대화 데이터를 모델이 학습해야 한다고 가정합니다.  
사용자가 **"안녕하세요, 오늘 날씨는 어떤가요?"**라고 물었고,  
모델은 **"안녕하세요! 오늘 날씨는 맑고 화창합니다."**라고 응답해야 한다고 합시다.

이 데이터를 학습하기 위해 먼저 전체 대화 데이터를 정수로 인코딩합니다.  
토크나이저는 대화의 구조를 구분하기 위해 `<|im_start|>`와 `<|im_end|>`을 사용하여 정수 시퀀스를 생성한다고 가정해봅시다.  
(실제로는 LLM 템플릿이 이보다는 복잡함을 기억하고 혼동하지 맙시다.)
정수 시퀀스는 다음과 같이 구성될 수 있습니다.  

```
<|im_start|>user 안녕하세요, 오늘 날씨는 어떤가요?<|im_end|>  
<|im_start|>assistant 안녕하세요! 오늘 날씨는 맑고 화창합니다.<|im_end|>
```

이를 정수로 변환하면 다음과 같습니다.  
`input_ids = [1001, 2001, 3001, 4001, 5001, 6001, 7001, 1002, 1001, 8001, 9001, 1003, 2002]`  
모델이 예측해야 하는 부분은 `assistant`의 응답인 "안녕하세요! 오늘 날씨는 맑고 화창합니다."입니다. 따라서, `labels`는 다음과 같이 설정됩니다.

`labels = [-100, -100, -100, -100, -100, -100, -100, -100, -100, 8001, 9001, 1003, 2002]`  

이처럼 `labels`는 모델의 출력이 필요한 영역만을 포함하고, 나머지 부분은 `-100`으로 채워져 모델이 실제로 예측하고 오차를 계산해야 하는 대상(학습 대상)에서 제외됩니다. 이를 통해 모델은 불필요한 영역을 학습하지 않고, 필요한 응답 영역에만 집중할 수 있습니다.


## 5. 학습

In [24]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

In [25]:
# 학습 시작
trainer.train()   # 모델이 자동으로 허브와 output_dir에 저장됨

# 모델 저장
trainer.save_model()   # 최종 모델을 저장

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,1.083300
20,0.575100
30,0.418100
40,0.357000
50,0.292400
60,0.280800
70,0.227400
80,0.197300
90,0.158600
100,0.177000


## 6. 테스트 데이터 준비

실제 모델에 입력을 넣을 때에는 입력의 뒤에 '<|im_start|>assistant'가 부착되어서 넣는 것이 좋습니다. 그래야만 모델이 바로 답변을 생성합니다.

In [26]:
prompt_lst = []
label_lst = []

for prompt in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(
        prompt, tokenize=False, add_generation_prompt=False
    )
    input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = text.split('<|im_start|>assistant')[1]
    prompt_lst.append(input)
    label_lst.append(label)

In [27]:
print(prompt_lst[42])

<|im_start|>system

당신은 SQL을 생성하는 AI 모델입니다.
아래는 데이터베이스 스키마(DDL)입니다.

<SCHEMA>
-- 작업완료로그 테이블
CREATE TABLE IF NOT EXISTS `log_lottranslog` (
  `TRANSLOGID` bigint(20) NOT NULL DEFAULT 0 COMMENT '로그ID',
  `LOTNO` char(20) NOT NULL DEFAULT '' COMMENT 'LOT번호',
  `LINENO` char(20) DEFAULT NULL COMMENT '생산라인번호',
  `TRANSACTIONNAME` char(50) NOT NULL DEFAULT '' COMMENT '처리명',
  `TIMELOGGED` datetime(3) NOT NULL DEFAULT '0000-00-00 00:00:00.000' COMMENT '로그입력일시',
  `ACTUALTIME` datetime(3) NOT NULL DEFAULT '0000-00-00 00:00:00.000' COMMENT '실제실행일시',
  `MATERIALCODE` char(30) NOT NULL DEFAULT '' COMMENT '자재코드',
  `MATERIALNAME` char(50) NOT NULL DEFAULT '' COMMENT '자재명',
  `TRANSQTY` double NOT NULL DEFAULT 0 COMMENT '변경수량',
  `CURRENTQTY` double NOT NULL DEFAULT 0 COMMENT '현재수량',
  `NEXTQTY` double DEFAULT NULL COMMENT '변경반영된수량',
  `TRANSUOM` char(5) DEFAULT NULL COMMENT '측정단위',
  `WAREHOUSECODE` char(20) DEFAULT NULL COMMENT '창고코드',
  `BOPMATERIALCODE` char(30) DEFAULT NULL COMMENT '자재코드',
  `

In [28]:
print(label_lst[42])


SELECT f.PROCESSCODE, AVG(TIMESTAMPDIFF(SECOND, c.create_time, f.ACTUALTIME)) AS AVG_CYCLE_SEC FROM LOG_LOTTRANSLOG f LEFT JOIN (SELECT LOTNO, MIN(ACTUALTIME) AS create_time FROM LOG_LOTTRANSLOG WHERE TRANSACTIONNAME='CREATE' GROUP BY LOTNO) c ON c.LOTNO = f.LOTNO WHERE f.LINENO='ML0000' AND f.TRANSACTIONNAME='FINISH' AND DATE_FORMAT(f.ACTUALTIME, '%Y%m') = DATE_FORMAT(CURDATE(), '%Y%m') GROUP BY f.PROCESSCODE;<|im_end|>



## 7. 파인 튜닝 모델 테스트

`AutoPeftModelForCausalLM()`의 입력으로 LoRA Adapter가 저장된 체크포인트의 주소를 넣으면 LoRA Adapter가 기존의 LLM과 부착되어 로드됩니다. 이 과정은 LoRA Adapter의 가중치를 사전 학습된 언어 모델(LLM)에 통합하여 미세 조정된 모델을 완성하는 것을 의미합니다.

`peft_model_id` 변수는 미세 조정된 가중치가 저장된 체크포인트의 경로를 나타냅니다. `"qwen2-7b-rag-ko/checkpoint-285"`는 LoRA Adapter 가중치가 저장된 위치로, 이 경로에서 해당 가중치를 불러옵니다.

`fine_tuned_model`은 `AutoPeftModelForCausalLM.from_pretrained` 메서드를 통해 체크포인트를 로드하여 생성됩니다. 이 메서드는 LLM과 LoRA Adapter를 결합하고, 최적화된 설정으로 모델을 메모리에 로드합니다. `device_map="auto"` 옵션은 모델을 자동으로 GPU에 배치합니다.

`pipeline`은 Hugging Face의 고수준 유틸리티로, NLP 작업(예: 텍스트 생성, 번역, 요약 등)을 간단히 수행할 수 있게 해줍니다. 이 코드에서 사용된 `pipeline("text-generation")`은 텍스트 생성 작업을 수행하기 위한 파이프라인 객체를 생성합니다. 파이프라인은 내부적으로 모델과 토크나이저를 관리하여, 입력 텍스트를 토큰화하고, 모델을 통해 생성된 결과를 다시 디코딩하여 사람이 읽을 수 있는 텍스트로 변환합니다.

이 코드는 미세 조정된 LLM을 로드한 뒤, 이를 이용해 텍스트 생성 작업을 간단히 수행할 수 있도록 준비하는 데 목적이 있습니다. `pipeline`을 통해 텍스트 생성 작업을 실행하면, 입력 텍스트에 기반하여 모델이 다음 토큰을 예측하고 이를 반복적으로 생성합니다. 이 과정은 사용자에게 자연스러운 텍스트를 출력하는 데 사용됩니다.

In [29]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import  AutoTokenizer, pipeline

In [31]:
peft_model_id = "qwen3-4b-text-to-sql-ko/checkpoint-450"
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [32]:
eos_token = tokenizer("<|im_end|>",add_special_tokens=False)["input_ids"][0]

In [33]:
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()

In [34]:
for prompt, label in zip(prompt_lst[0:10], label_lst[0:10]):
    # print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


    response:
WITH MONTHLY AS (SELECT DATE_FORMAT(ACTUALTIME, '%Y%m') AS YM, SUM(TRANSQTY) AS TOTAL_QTY FROM LOG_LOTTRANSLOG WHERE LINENO='ML0000' AND TRANSACTIONNAME='FINISH' AND DATE_FORMAT(ACTUALTIME, '%Y')=DATE_FORMAT(CURDATE(), '%Y') GROUP BY DATE_FORMAT(ACTUALTIME, '%Y%m')) SELECT YM, TOTAL_QTY, (SELECT SUM(CREATEQTY) FROM VW_LOT WHERE WORKSTATE='FAILED' AND DATE_FORMAT(CREATEDATE, '%Y')=DATE_FORMAT(CURDATE(), '%Y') AND DATE_FORMAT(CREATEDATE, '%Y%m')=YM) AS FAIL_QTY, ROUND((SELECT SUM(CREATEQTY) FROM VW_LOT WHERE WORKSTATE='FAILED' AND DATE_FORMAT(CREATEDATE, '%Y')=DATE_FORMAT(CURDATE(), '%Y') AND DATE_FORMAT(CREATEDATE, '%Y%m')=YM) / NULLIF(TOTAL_QTY, 0) * 1000000, 2) AS DEFECT_RATE_PPM FROM MONTHLY ORDER BY YM;
    label:

SELECT DATE_FORMAT(A.ACTUALTIME,'%Y%m') AS ym, ROUND((SUM(COALESCE(fl.fail_qty,0))/NULLIF(SUM(A.TRANSQTY),0))*1000000,2) AS ppm FROM LOG_LOTTRANSLOG A LEFT JOIN (SELECT LOTNO, MATERIALCODE, SUM(QUEUEDQTY) AS fail_qty FROM VW_LOT WHERE WORKSTATE='FAILED' AND 

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


    response:
SELECT SUM(TRANSQTY) AS FINISHSUM FROM LOG_LOTTRANSLOG WHERE LINENO = 'ML0000' AND TRANSACTIONNAME = 'FINISH' AND ACTUALTIME >= '2025-03-16 00:00:00.000' AND ACTUALTIME < '2025-04-01 00:00:00.000';
    label:

SELECT SUM(TRANSQTY) AS FINISHSUM FROM LOG_LOTTRANSLOG WHERE LINENO = 'ML0000' AND TRANSACTIONNAME = 'FINISH' AND DATE(ACTUALTIME) BETWEEN '2025-03-16' AND '2025-03-31';<|im_end|>

--------------------------------------------------


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

    response:
SELECT fd.ymd AS '작업일자', SUM(fd.prod_qty) AS '총 생산수량', SUM(COALESCE(fl.fail_qty,0)) AS '총 불량수량', ROUND((SUM(COALESCE(fl.fail_qty,0))/NULLIF(SUM(fd.prod_qty),0))*1000000,2) AS '불량율(ppm)' FROM ( SELECT A.LOTNO, A.MATERIALCODE, DATE_FORMAT(A.ACTUALTIME,'%Y%m%d') AS ymd, SUM(A.TRANSQTY) AS prod_qty FROM LOG_LOTTRANSLOG A WHERE A.TRANSACTIONNAME='FINISH' AND A.LINENO='ML0000' AND DATE_FORMAT(A.ACTUALTIME,'%Y%m') = DATE_FORMAT(CURDATE(),'%Y%m') GROUP BY A.LOTNO, A.MATERIALCODE, DATE_FORMAT(A.ACTUALTIME,'%Y%m%d') ) fd LEFT JOIN ( SELECT LOTNO, MATERIALCODE, SUM(QUEUEDQTY) AS fail_qty FROM VW_LOT WHERE WORKSTATE='FAILED' AND DATE_FORMAT(CREATEDATE,'%Y%m') = DATE_FORMAT(CURDATE(),'%Y%m') GROUP BY LOTNO, MATERIALCODE ) fl ON fl.LOTNO=fd.LOTNO AND fl.MATERIALCODE=fd.MATERIALCODE GROUP BY fd.ymd ORDER BY SUM(COALESCE(fl.fail_qty,0)) DESC LIMIT 1;
    label:

SELECT fd.ymd AS '작업일자', SUM(fd.day_prod_qty) AS '총 생산수량', SUM(COALESCE(fl.fail_qty,0)) AS '총 불량수량', ROUND((SUM(COALESCE(fl.fai

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


    response:
SELECT DATE_FORMAT(sw.STARTEDTIME, '%Y%m%d') AS WORKDATE, HOUR(sw.STARTEDTIME) AS HOUR_OF_DAY, SUM(sw.STARTEDQTY) AS WIP_START_QTY FROM sgt_wiplot sw WHERE sw.LINENO = 'ML0000' AND sw.WORKSTATE = 'PROCESSING' AND DATE(sw.STARTEDTIME) = '2025-05-20' GROUP BY DATE_FORMAT(sw.STARTEDTIME, '%Y%m%d'), HOUR(sw.STARTEDTIME) ORDER BY HOUR_OF_DAY;
    label:

SELECT HOUR(sw.STARTEDTIME) AS WORK_HOUR, COUNT(DISTINCT sw.LOTNO) AS LOT_CNT, SUM(sw.STARTEDQTY) AS WIP_QTY FROM sgt_wiplot sw WHERE sw.LINENO = 'ML0000' AND sw.WORKSTATE = 'PROCESSING' AND DATE(sw.STARTEDTIME) = '2025-05-20' GROUP BY HOUR(sw.STARTEDTIME) ORDER BY WORK_HOUR;<|im_end|>

--------------------------------------------------


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


    response:
WITH CYCLE_DATA AS (SELECT f.LOTNO, TIMESTAMPDIFF(SECOND,c.create_time,f.ACTUALTIME) AS CYCLE_SEC FROM LOG_LOTTRANSLOG f LEFT JOIN (SELECT LOTNO,MIN(ACTUALTIME) AS create_time FROM LOG_LOTTRANSLOG WHERE TRANSACTIONNAME='CREATE' GROUP BY LOTNO) c ON c.LOTNO=f.LOTNO WHERE f.LINENO='ML0000' AND f.TRANSACTIONNAME='FINISH') SELECT CYCLE_SEC, COUNT(*) AS cnt FROM CYCLE_DATA GROUP BY CYCLE_SEC ORDER BY CYCLE_SEC;
    label:

WITH CYCLE_DATA AS (SELECT TIMESTAMPDIFF(SECOND,c.create_time,f.ACTUALTIME) AS CYCLE_SEC FROM LOG_LOTTRANSLOG f LEFT JOIN (SELECT LOTNO,MIN(ACTUALTIME) AS create_time FROM LOG_LOTTRANSLOG WHERE TRANSACTIONNAME='CREATE' GROUP BY LOTNO) c ON c.LOTNO=f.LOTNO WHERE f.LINENO='ML0000' AND f.TRANSACTIONNAME='FINISH') SELECT CASE WHEN CYCLE_SEC < 60 THEN '0-60s' WHEN CYCLE_SEC < 120 THEN '60-120s' WHEN CYCLE_SEC < 180 THEN '120-180s' WHEN CYCLE_SEC < 300 THEN '180-300s' WHEN CYCLE_SEC < 600 THEN '300-600s' ELSE '600s+' END AS CYCLE_RANGE, COUNT(*) AS LOT_COUNT FROM 

## 8. 기본 모델 테스트

이번에는 LoRA Adapter를 merge하지 않은 기본 모델로 테스트 데이터에 대해서 인퍼런스해보겠습니다.

In [35]:
base_model_id = "Qwen/Qwen3-4B-Instruct-2507"
model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [36]:
for prompt, label in zip(prompt_lst[0:10], label_lst[0:10]):
    # print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


    response:
-- 요청하신 '올해 월별 전체 불량률(ppm)' 추이를 보여주는 쿼리는, 현재 제공된 스키마에 불량률 관련 데이터(예: 불량 수, 총 생산 수량, 불량 발생일시 등)가 포함되어 있지 않기 때문에, 불량률을 계산할 수 없습니다.

불량률(ppm)을 계산하기 위해서는 다음과 같은 정보가 필요합니다:
- 불량 발생 수
- 총 생산 수량
- 생산 일시 또는 월별 분류

이러한 데이터는 제공된 테이블들(`log_lottranslog`, `vw_lot`, `udt_material`, `udt_process`, `udt_factoryline` 등)에 포함되어 있지 않으며, 불량 관련 로그나 품질 데이터 테이블이 없습니다.

따라서, 요청하신 쿼리는 **데이터가 부족하여 생성할 수 없습니다**.
    label:

SELECT DATE_FORMAT(A.ACTUALTIME,'%Y%m') AS ym, ROUND((SUM(COALESCE(fl.fail_qty,0))/NULLIF(SUM(A.TRANSQTY),0))*1000000,2) AS ppm FROM LOG_LOTTRANSLOG A LEFT JOIN (SELECT LOTNO, MATERIALCODE, SUM(QUEUEDQTY) AS fail_qty FROM VW_LOT WHERE WORKSTATE='FAILED' AND DATE_FORMAT(CREATEDATE,'%Y')=DATE_FORMAT(CURDATE(),'%Y') GROUP BY LOTNO, MATERIALCODE) fl ON A.LOTNO=fl.LOTNO AND A.MATERIALCODE=fl.MATERIALCODE WHERE A.TRANSACTIONNAME='FINISH' AND DATE_FORMAT(A.ACTUALTIME,'%Y')=DATE_FORMAT(CURDATE(),'%Y') GROUP BY DATE_FORMAT(A.ACTUALTIME,'%Y%m') ORDER BY ym;<|im_end|>

----------------------

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


    response:
SELECT 
    SUM(l.createqty) AS cumulative_production
FROM 
    vw_lot l
    JOIN udt_factoryline f ON l.lineno = f.lineno
WHERE 
    f.lineno = 'PORCHE' 
    AND l.createdate >= '2025-03-16 00:00:00.000'
    AND l.createdate < '2025-03-32 00:00:00.000';
    label:

SELECT SUM(TRANSQTY) AS FINISHSUM FROM LOG_LOTTRANSLOG WHERE LINENO = 'ML0000' AND TRANSACTIONNAME = 'FINISH' AND DATE(ACTUALTIME) BETWEEN '2025-03-16' AND '2025-03-31';<|im_end|>

--------------------------------------------------


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

    response:
SELECT 
    DATE(TIMELOGGED) AS defect_day
FROM 
    log_lottranslog
WHERE 
    TRANSACTIONNAME LIKE '%불량%'
GROUP BY 
    DATE(TIMELOGGED)
ORDER BY 
    COUNT(*) DESC
LIMIT 1;
    label:

SELECT fd.ymd AS '작업일자', SUM(fd.day_prod_qty) AS '총 생산수량', SUM(COALESCE(fl.fail_qty,0)) AS '총 불량수량', ROUND((SUM(COALESCE(fl.fail_qty,0))/NULLIF(SUM(fd.day_prod_qty),0))*100,2) AS '불량율 퍼센트(%)' FROM (SELECT A.LOTNO, A.MATERIALCODE, DATE_FORMAT(A.ACTUALTIME,'%Y%m%d') AS ymd, SUM(A.TRANSQTY) AS day_prod_qty FROM LOG_LOTTRANSLOG A WHERE A.TRANSACTIONNAME='FINISH' AND DATE_FORMAT(A.ACTUALTIME,'%Y%m')=DATE_FORMAT(CURDATE(),'%Y%m') GROUP BY A.LOTNO, A.MATERIALCODE, DATE_FORMAT(A.ACTUALTIME,'%Y%m%d')) fd LEFT JOIN (SELECT A.LOTNO, A.MATERIALCODE, SUM(A.QUEUEDQTY) AS fail_qty FROM VW_LOT A INNER JOIN UDT_MATERIAL SC ON A.MATERIALCODE=SC.MATERIALCODE WHERE A.WORKSTATE='FAILED' AND DATE_FORMAT(A.CREATEDATE,'%Y%m')=DATE_FORMAT(CURDATE(),'%Y%m') GROUP BY A.LOTNO, A.MATERIALCODE) fl ON fd.LOTNO=fl.LOTN

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


    response:
SELECT 
    HOUR(t.STARTEDTIME) AS hour,
    SUM(t.STARTEDQTY) AS started_qty
FROM 
    sgt_wiplot t
JOIN 
    udt_factoryline f ON t.LINENO = f.LINENO
WHERE 
    f.LINENO LIKE '%포르쉐%'
    AND t.STARTEDTIME >= '2025-05-20 00:00:00'
    AND t.STARTEDTIME < '2025-05-21 00:00:00'
GROUP BY 
    HOUR(t.STARTEDTIME)
ORDER BY 
    hour;
    label:

SELECT HOUR(sw.STARTEDTIME) AS WORK_HOUR, COUNT(DISTINCT sw.LOTNO) AS LOT_CNT, SUM(sw.STARTEDQTY) AS WIP_QTY FROM sgt_wiplot sw WHERE sw.LINENO = 'ML0000' AND sw.WORKSTATE = 'PROCESSING' AND DATE(sw.STARTEDTIME) = '2025-05-20' GROUP BY HOUR(sw.STARTEDTIME) ORDER BY WORK_HOUR;<|im_end|>

--------------------------------------------------


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


    response:
SELECT 
    l.LOTNO,
    l.LINENO,
    l.TRANSACTIONNAME,
    l.ACTUALTIME,
    l.MATERIALCODE,
    l.MATERIALNAME,
    l.TRANSQTY,
    l.CURRENTQTY,
    l.NEXTQTY,
    l.TRANSUOM,
    l.WAREHOUSECODE,
    l.BOPMATERIALCODE,
    l.PROCESSCODE,
    l.USERCODE,
    (julianday(l.ACTUALTIME) - julianday(l.TIMELOGGED)) * 24 * 60 AS cycle_time_minutes
FROM 
    log_lottranslog l
JOIN 
    udt_factoryline f ON l.LINENO = f.LINENO
WHERE 
    f.LINENO LIKE 'PORCHE%' 
    AND l.TRANSACTIONNAME = 'LOT PROCESSING'
ORDER BY 
    cycle_time_minutes DESC;
    label:

WITH CYCLE_DATA AS (SELECT TIMESTAMPDIFF(SECOND,c.create_time,f.ACTUALTIME) AS CYCLE_SEC FROM LOG_LOTTRANSLOG f LEFT JOIN (SELECT LOTNO,MIN(ACTUALTIME) AS create_time FROM LOG_LOTTRANSLOG WHERE TRANSACTIONNAME='CREATE' GROUP BY LOTNO) c ON c.LOTNO=f.LOTNO WHERE f.LINENO='ML0000' AND f.TRANSACTIONNAME='FINISH') SELECT CASE WHEN CYCLE_SEC < 60 THEN '0-60s' WHEN CYCLE_SEC < 120 THEN '60-120s' WHEN CYCLE_SEC < 180 THEN '120-180